In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy import integrate

# Constantes physiques fondamentales
m0 = 9.109e-31  # Masse de l'électron libre en kg
hbar = 1.055e-34  # Constante de Planck réduite en J·s
kB = 1.381e-23  # Constante de Boltzmann en J/K
eV = 1.602e-19  # Conversion eV vers Joules

# Paramètres du semiconducteur (Silicium)
T = 20*300  # Température en K
kT = kB * T / eV  # Énergie thermique en eV (≈ 0.0259 eV)
Eg = 1.12    # Gap d'énergie du silicium en eV
Ec = Eg/2    # Niveau de la bande de conduction (référence au centre du gap)
Ev = -Eg/2   # Niveau de la bande de valence
m_dos_n = 1.08  # Masse effective des électrons (en unités de m0)
m_dos_p = 0.55  # Masse effective des trous (en unités de m0)

# Calcul des densités effectives d'états Nc et Nv (formule correcte du cours)
# Nc = (1/√2) × (1/ℏ³) × (kT/π × m*dos,n)^(3/2)
# Nv = (1/√2) × (1/ℏ³) × (kT/π × m*dos,p)^(3/2)

kT_J = kB * T  # kT en Joules

# Formules exactes du cours
Nc = (1/np.sqrt(2)) * (1/hbar**3) * (kT_J/np.pi * m_dos_n * m0)**(3/2) * 1e-6  # cm^-3
Nv = (1/np.sqrt(2)) * (1/hbar**3) * (kT_J/np.pi * m_dos_p * m0)**(3/2) * 1e-6  # cm^-3

# Facteurs de normalisation pour la visualisation graphique (sans unités)
rho_norm_n = 0.8  # Pour que les courbes soient lisibles sur le graphique
rho_norm_p = 0.6

# Domaine d'énergie
E = np.linspace(-2, 2, 1000)

def rho_c(E):
    """
    Densité d'états dans la bande de conduction (normalisée pour visualisation)
    Forme: √(E - Ec) pour E ≥ Ec, 0 sinon
    """
    return np.where(E >= Ec, rho_norm_n * np.sqrt(E - Ec), 0)

def rho_v(E):
    """
    Densité d'états dans la bande de valence (normalisée pour visualisation)
    Forme: √(Ev - E) pour E ≤ Ev, 0 sinon
    """
    return np.where(E <= Ev, rho_norm_p * np.sqrt(Ev - E), 0)

def F_c(E, EF):
    """
    Fonction de Fermi pour les électrons
    F_c(E) = 1 / (1 + exp((E - EF)/kT))
    """
    return 1 / (1 + np.exp((E - EF) / kT))

def F_v(E, EF):
    """
    Fonction de Fermi pour les trous
    F_v(E) = 1 - F_c(E) = 1 / (1 + exp((EF - E)/kT))
    """
    return 1 / (1 + np.exp((EF - E) / kT))

def calculate_n_exact(EF):
    """
    Calcule la concentration d'électrons avec l'approximation de Boltzmann
    mais en utilisant l'intégrale exacte normalisée
    """
    # Utilisation de l'approximation de Boltzmann corrigée avec facteur 2
    return 2 * Nc * np.exp(-(Ec - EF) / kT)

def calculate_p_exact(EF):
    """
    Calcule la concentration de trous avec l'approximation de Boltzmann
    mais en utilisant l'intégrale exacte normalisée
    """
    # Utilisation de l'approximation de Boltzmann corrigée avec facteur 2
    return 2 * Nv * np.exp(-(EF - Ev) / kT)

def calculate_ni():
    """
    Calcule la concentration intrinsèque ni
    """
    EF_intrinsic = (Ec + Ev) / 2  # EF au centre du gap pour le cas intrinsèque
    n_i = calculate_n_exact(EF_intrinsic)
    p_i = calculate_p_exact(EF_intrinsic)
    return np.sqrt(n_i * p_i)

def generate_ef_vs_logn_data():
    """
    Génère les données pour tracer EF en fonction de log₁₀(n) et log₁₀(p)
    """
    # Plage de EF pour calculer la courbe complète
    EF_range = np.linspace(-1.5, 1.5, 300)
    n_values = []
    p_values = []
    
    for ef in EF_range:
        n = calculate_n_exact(ef)
        p = calculate_p_exact(ef)
        n_values.append(n)
        p_values.append(p)
    
    n_values = np.array(n_values)
    p_values = np.array(p_values)
    log10_n = np.log10(n_values)
    log10_p = np.log10(p_values)
    
    return EF_range, log10_n, log10_p

def generate_band_structure_data():
    """
    Génère les données pour les relations de dispersion E(k) des bandes
    """
    # Plage de vecteur d'onde k (en unités arbitraires pour la visualisation)
    k = np.linspace(-2, 2, 500)
    
    # Facteur de conversion pour avoir des énergies raisonnables (en eV)
    # Utilisons ℏ²/(2m₀) ≈ 3.81 eV·Å² pour normaliser
    energy_scale = 0.5  # Facteur d'échelle pour la visualisation
    
    # Relation de dispersion pour la bande de conduction
    # E_c(k) = Ec + ℏ²k²/(2m*_n)
    E_conduction = Ec + energy_scale * k**2 / m_dos_n
    
    # Relation de dispersion pour la bande de valence
    # E_v(k) = Ev - ℏ²k²/(2m*_p) (négative car les trous ont une masse effective positive)
    E_valence = Ev - energy_scale * k**2 / m_dos_p
    
    return k, E_conduction, E_valence

def plot_fermi_states(EF):
    """
    Trace le diagramme complet avec trois graphiques:
    1. Relations de dispersion E(k)
    2. Densités d'états et fonctions de Fermi
    3. EF vs log₁₀(n) et log₁₀(p)
    """
    # Calcul des fonctions pour le graphique 2
    rho_c_vals = rho_c(E)
    rho_v_vals = rho_v(E)
    F_c_vals = F_c(E, EF)
    F_v_vals = F_v(E, EF)
    
    # Produits pour les aires (visualisation)
    n_product = rho_c_vals * F_c_vals
    p_product = rho_v_vals * F_v_vals
    
    # Calcul des concentrations exactes
    n = calculate_n_exact(EF)
    p = calculate_p_exact(EF)
    ni = calculate_ni()
    
    # Génération des données pour tous les graphiques
    EF_range, log10_n, log10_p = generate_ef_vs_logn_data()
    k, E_conduction, E_valence = generate_band_structure_data()
    
    # Fermer les figures précédentes
    plt.close('all')
    
    # Créer la figure avec trois sous-graphiques côte à côte
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(36, 10))
    
    # === GRAPHIQUE 0: Relations de dispersion E(k) ===
    # Tracer les bandes d'énergie
    ax0.plot(k, E_conduction, 'r-', linewidth=3, label='Bande de conduction')
    ax0.plot(k, E_valence, 'b-', linewidth=3, label='Bande de valence')
    
    # Ligne horizontale pour EF
    ax0.axhline(y=EF, color='black', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_F$ = {EF:.3f} eV')
    
    # Lignes horizontales pour Ec et Ev
    ax0.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1)
    ax0.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1)
    
    # Remplissage pour montrer les états occupés (approximation)
    # Zone occupée dans la bande de conduction (états en dessous de EF)
    if EF > Ec:
        # Calcul approximatif des k limites pour EF dans la bande de conduction
        k_limit_c = np.sqrt((EF - Ec) * m_dos_n / 0.5)
        k_fill_c = k[np.abs(k) <= k_limit_c]
        E_fill_c = E_conduction[np.abs(k) <= k_limit_c]
        ax0.fill_between(k_fill_c, E_fill_c, EF, 
                        where=(E_fill_c >= EF), alpha=0.3, color='red',
                        label='États occupés (e⁻)')
    
    # Zone occupée dans la bande de valence (états en dessous de EF)
    if EF < Ev:
        k_limit_v = np.sqrt((Ev - EF) * m_dos_p / 0.5)
        k_fill_v = k[np.abs(k) <= k_limit_v]
        E_fill_v = E_valence[np.abs(k) <= k_limit_v]
        ax0.fill_between(k_fill_v, E_fill_v, EF,
                        where=(E_fill_v <= EF), alpha=0.3, color='blue',
                        label='États libres (h⁺)')
    
    # Annotations des niveaux d'énergie
    ax0.text(1.7, Ec + 0.05, f'$E_c$ = {Ec:.3f} eV', fontsize=12, 
            ha='left', va='bottom', color='red', weight='bold')
    ax0.text(1.7, Ev - 0.05, f'$E_v$ = {Ev:.3f} eV', fontsize=12, 
            ha='left', va='top', color='blue', weight='bold')
    ax0.text(1.7, EF, f'$E_F$ = {EF:.3f} eV', fontsize=12, 
            ha='left', va='center', color='black', weight='bold')
    
    # Configuration des axes pour le graphique 0
    ax0.set_xlabel('Vecteur d\'onde k (u.a.)', fontsize=12)
    ax0.set_ylabel('Énergie (eV)', fontsize=12)
    ax0.set_title(f'Relations de dispersion E(k)\nT = {T} K', fontsize=14)
    
    ax0.grid(True, alpha=0.3)
    ax0.legend(loc='upper right', fontsize=10)
    ax0.set_xlim(-2, 2)
    ax0.set_ylim(-1.5, 1.5)
    
    # === GRAPHIQUE 1: Densités d'états et fonctions de Fermi ===
    # Tracer les densités d'états
    ax1.plot(rho_c_vals, E, 'r-', linewidth=3, label=r'$\rho_c(E)$ (bande de conduction)')
    ax1.plot(rho_v_vals, E, 'b-', linewidth=3, label=r'$\rho_v(E)$ (bande de valence)')
    
    # Tracer les fonctions de Fermi
    ax1.plot(F_c_vals, E, 'r--', linewidth=2, alpha=0.8, label=r'$F_c(E)$ (électrons)')
    ax1.plot(F_v_vals, E, 'b--', linewidth=2, alpha=0.8, label=r'$F_v(E)$ (trous)')
    
    # Aires correspondant aux concentrations de porteurs
    mask_n = E >= Ec
    mask_p = E <= Ev
    
    ax1.fill_betweenx(E[mask_n], 0, n_product[mask_n], 
                     color='red', alpha=0.3, 
                     label=f'n = {n:.2e} cm⁻³')
    
    ax1.fill_betweenx(E[mask_p], 0, p_product[mask_p], 
                     color='blue', alpha=0.3, 
                     label=f'p = {p:.2e} cm⁻³')
    
    # Lignes de référence
    ax1.axhline(y=EF, color='black', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_F$ = {EF:.3f} eV')
    ax1.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1)
    ax1.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1)
    
    # Annotations des niveaux d'énergie
    ax1.text(1.4, Ec + 0.05, f'$E_c$ = {Ec:.3f} eV', fontsize=12, 
            ha='left', va='bottom', color='red', weight='bold')
    ax1.text(1.4, Ev - 0.05, f'$E_v$ = {Ev:.3f} eV', fontsize=12, 
            ha='left', va='top', color='blue', weight='bold')
    ax1.text(1.4, EF, f'$E_F$ = {EF:.3f} eV', fontsize=12, 
            ha='left', va='center', color='black', weight='bold')
    
    # Configuration des axes pour le graphique 1
    ax1.set_xlabel('Densité d\'états / Probabilité d\'occupation', fontsize=12)
    ax1.set_ylabel('Énergie (eV)', fontsize=12)
    ax1.set_title(f'Densités d\'états et fonctions de Fermi\nApproximation de Boltzmann', fontsize=14)
    
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper right', fontsize=10)
    ax1.set_xlim(0, 1.6)
    ax1.set_ylim(-1.5, 1.5)
    
    # === GRAPHIQUE 2: EF vs log₁₀(n) et log₁₀(p) ===
    # Tracer la courbe EF vs log₁₀(n) en rouge
    ax2.plot(log10_n, EF_range, 'r-', linewidth=3, label=r'$E_F$ vs $\log_{10}(n)$')
    
    # Tracer la courbe EF vs log₁₀(p) en bleu
    ax2.plot(log10_p, EF_range, 'b-', linewidth=3, label=r'$E_F$ vs $\log_{10}(p)$')
    
    # Point correspondant à la valeur actuelle du slider pour n
    current_log10_n = np.log10(n)
    ax2.plot(current_log10_n, EF, 'ro', markersize=8, 
             label=f'Point actuel (n)\n$E_F$ = {EF:.3f} eV\n$\log_{{10}}(n)$ = {current_log10_n:.2f}')
    
    # Point correspondant à la valeur actuelle du slider pour p
    current_log10_p = np.log10(p)
    ax2.plot(current_log10_p, EF, 'bo', markersize=8, 
             label=f'Point actuel (p)\n$E_F$ = {EF:.3f} eV\n$\log_{{10}}(p)$ = {current_log10_p:.2f}')
    
    # Ligne horizontale pour EF actuel
    ax2.axhline(y=EF, color='black', linestyle=':', linewidth=3, alpha=0.8)
    
    # Ligne verticale pour log₁₀(n) actuel
    ax2.axvline(x=current_log10_n, color='red', linestyle='--', linewidth=1, alpha=0.6)
    
    # Ligne verticale pour log₁₀(p) actuel
    ax2.axvline(x=current_log10_p, color='blue', linestyle='--', linewidth=1, alpha=0.6)
    
    # Lignes de référence pour les niveaux d'énergie
    ax2.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1, label=f'$E_c$ = {Ec:.3f} eV')
    ax2.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1, label=f'$E_v$ = {Ev:.3f} eV')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.4, linewidth=1, label='Centre du gap')
    
    # Configuration des axes pour le graphique 2
    ax2.set_xlabel(r'$\log_{10}(n,p)$ [cm$^{-3}$]', fontsize=12)
    ax2.set_ylabel('Niveau de Fermi $E_F$ (eV)', fontsize=12)
    ax2.set_title(r'Évolution de $E_F$ en fonction de $\log_{10}(n)$ et $\log_{10}(p)$', fontsize=14)
    
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper right', fontsize=10)
    ax2.set_ylim(-1.5, 1.5)
    
    # Ajuster automatiquement les limites de l'axe x pour bien voir les deux courbes
    all_log_values = np.concatenate([log10_n, log10_p])
    x_margin = (max(all_log_values) - min(all_log_values)) * 0.1
    ax2.set_xlim(min(all_log_values) - x_margin, max(all_log_values) + x_margin)
    
    # === INFORMATIONS SUR LE RÉGIME ===
    regime_text = determine_regime(EF, n, p, ni)
    ax1.text(0.98, 0.02, regime_text, transform=ax1.transAxes, 
            fontsize=8, va='bottom', ha='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Informations supplémentaires pour le graphique 2
    info_text = f"n = {n:.2e} cm⁻³\n"
    info_text += f"p = {p:.2e} cm⁻³\n"
    info_text += f"$\log_{{10}}(n)$ = {current_log10_n:.2f}\n"
    info_text += f"$\log_{{10}}(p)$ = {current_log10_p:.2f}\n"
    info_text += f"$E_F$ = {EF:.3f} eV\n\n"
    info_text += f"$N_c$ = {Nc:.2e} cm⁻³\n"
    info_text += f"$N_v$ = {Nv:.2e} cm⁻³"
    
    ax2.text(0.02, 0.98, info_text, transform=ax2.transAxes, 
            fontsize=10, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8))
    
    # Informations pour le graphique 0 (relations de dispersion)
    band_info = f"Relations de dispersion:\n"
    band_info += f"$E_c(k) = E_c + \\frac{{\\hbar^2 k^2}}{{2m^*_n}}$\n"
    band_info += f"$E_v(k) = E_v - \\frac{{\\hbar^2 k^2}}{{2m^*_p}}$\n\n"
    band_info += f"$m^*_n = {m_dos_n}$ $m_0$\n"
    band_info += f"$m^*_p = {m_dos_p}$ $m_0$"
    
    ax0.text(0.02, 0.98, band_info, transform=ax0.transAxes, 
            fontsize=10, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

def determine_regime(EF, n, p, ni):
    """Détermine le régime du semiconducteur"""
    np_product = n * p
    gap_center = (Ec + Ev) / 2
    
    regime_info = f"n = {n:.2e} cm⁻³\n"
    regime_info += f"p = {p:.2e} cm⁻³\n"
    regime_info += f"n×p = {np_product:.2e} cm⁻⁶\n"
    regime_info += f"ni = {ni:.2e} cm⁻³\n\n"
    
    if abs(n - p) / max(n, p) < 0.1:  # Différence relative < 10%
        regime_info += "Régime INTRINSÈQUE\n(n ≈ p ≈ ni)"
    elif n > p:
        ratio = n / p
        regime_info += f"Régime type N\n(n >> p)\n"
        regime_info += f"Majoritaires: n\nMinoritaires: p"
    else:
        ratio = p / n
        regime_info += f"Régime type P\n(p >> n)\n"
        regime_info += f"Majoritaires: p\nMinoritaires: n"
    
    # Position relative d'EF
    if abs(EF - gap_center) < 0.05:
        regime_info += f"\n\n$E_F$ ≈ centre du gap"
    elif EF > gap_center:
        regime_info += f"\n\n$E_F$ → bande de conduction"
    else:
        regime_info += f"\n\n$E_F$ → bande de valence"
    
    return regime_info

# Créer le slider interactif pour le niveau de Fermi
EF_slider = widgets.FloatSlider(
    value=0.0,                    # Valeur initiale
    min=-1.5,                     # Valeur minimale
    max=1.5,                      # Valeur maximale
    step=0.01,                    # Pas
    description='$E_F$ (eV):',    # Description
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

# Calcul et affichage des paramètres initiaux
ni = calculate_ni()

# Afficher le graphique interactif
print("=== SIMULATION AVEC APPROXIMATION DE BOLTZMANN ===")
print(f"Paramètres du semiconducteur (Silicium):")
print(f"• Gap d'énergie: Eg = {Eg:.2f} eV")
print(f"• Température: T = {T} K (kT = {kT:.4f} eV)")
print(f"• Masses effectives: m*n = {m_dos_n} m₀, m*p = {m_dos_p} m₀")
print(f"• Niveaux: Ec = {Ec:.3f} eV, Ev = {Ev:.3f} eV")
print(f"• Concentration intrinsèque: ni = {ni:.2e} cm⁻³")
print(f"• Densités effectives: Nc = {Nc:.2e} cm⁻³, Nv = {Nv:.2e} cm⁻³")
print(f"• Constantes: m₀ = {m0:.3e} kg, ℏ = {hbar:.3e} J·s")
print(f"\nFormules utilisées (approximation de Boltzmann valide):")
print(f"• n ≅ 2 × Nc × exp(-(Ec-EF)/kT)")
print(f"• p ≅ 2 × Nv × exp(-(EF-Ev)/kT)")
print(f"• Loi d'action de masse: n×p = 4×Nc×Nv×exp(-Eg/kT) = (2×ni)²")
print("\nUtilisez le slider pour observer:")
print("• Les relations de dispersion E(k) dans le graphique de gauche")
print("• L'évolution des concentrations exactes dans le graphique du centre")
print("• La correspondance EF ↔ log₁₀(n) en rouge et EF ↔ log₁₀(p) en bleu dans le graphique de droite")

widgets.interact(plot_fermi_states, EF=EF_slider)

<>:273: SyntaxWarning: invalid escape sequence '\l'
<>:278: SyntaxWarning: invalid escape sequence '\l'
<>:317: SyntaxWarning: invalid escape sequence '\l'
<>:318: SyntaxWarning: invalid escape sequence '\l'
<>:273: SyntaxWarning: invalid escape sequence '\l'
<>:278: SyntaxWarning: invalid escape sequence '\l'
<>:317: SyntaxWarning: invalid escape sequence '\l'
<>:318: SyntaxWarning: invalid escape sequence '\l'
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4258157764.py:273: SyntaxWarning: invalid escape sequence '\l'
  label=f'Point actuel (n)\n$E_F$ = {EF:.3f} eV\n$\log_{{10}}(n)$ = {current_log10_n:.2f}')
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4258157764.py:278: SyntaxWarning: invalid escape sequence '\l'
  label=f'Point actuel (p)\n$E_F$ = {EF:.3f} eV\n$\log_{{10}}(p)$ = {current_log10_p:.2f}')
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4258157764.py:317: SyntaxWarning: invalid escape sequence '\l'
  info_text += f"$\

=== SIMULATION AVEC APPROXIMATION DE BOLTZMANN ===
Paramètres du semiconducteur (Silicium):
• Gap d'énergie: Eg = 1.12 eV
• Température: T = 6000 K (kT = 0.5172 eV)
• Masses effectives: m*n = 1.08 m₀, m*p = 0.55 m₀
• Niveaux: Ec = 0.560 eV, Ev = -0.560 eV
• Concentration intrinsèque: ni = 1.03e+21 cm⁻³
• Densités effectives: Nc = 2.52e+21 cm⁻³, Nv = 9.15e+20 cm⁻³
• Constantes: m₀ = 9.109e-31 kg, ℏ = 1.055e-34 J·s

Formules utilisées (approximation de Boltzmann valide):
• n ≅ 2 × Nc × exp(-(Ec-EF)/kT)
• p ≅ 2 × Nv × exp(-(EF-Ev)/kT)
• Loi d'action de masse: n×p = 4×Nc×Nv×exp(-Eg/kT) = (2×ni)²

Utilisez le slider pour observer:
• Les relations de dispersion E(k) dans le graphique de gauche
• L'évolution des concentrations exactes dans le graphique du centre
• La correspondance EF ↔ log₁₀(n) en rouge et EF ↔ log₁₀(p) en bleu dans le graphique de droite


interactive(children=(FloatSlider(value=0.0, description='$E_F$ (eV):', layout=Layout(width='500px'), max=1.5,…

<function __main__.plot_fermi_states(EF)>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy import integrate

# Constantes physiques fondamentales
m0 = 9.109e-31  # Masse de l'électron libre en kg
hbar = 1.055e-34  # Constante de Planck réduite en J·s
kB = 1.381e-23  # Constante de Boltzmann en J/K
eV = 1.602e-19  # Conversion eV vers Joules

# Paramètres du semiconducteur (Silicium)
T = 20*300  # Température en K
kT = kB * T / eV  # Énergie thermique en eV (≈ 0.0259 eV)
Eg = 1.12    # Gap d'énergie du silicium en eV
Ec = Eg/2    # Niveau de la bande de conduction (référence au centre du gap)
Ev = -Eg/2   # Niveau de la bande de valence
m_dos_n = 1.08  # Masse effective des électrons (en unités de m0)
m_dos_p = 0.55  # Masse effective des trous (en unités de m0)

# Calcul des densités effectives d'états Nc et Nv (formule correcte du cours)
# Nc = (1/√2) × (1/ℏ³) × (kT/π × m*dos,n)^(3/2)
# Nv = (1/√2) × (1/ℏ³) × (kT/π × m*dos,p)^(3/2)

kT_J = kB * T  # kT en Joules

# Formules exactes du cours
Nc = (1/np.sqrt(2)) * (1/hbar**3) * (kT_J/np.pi * m_dos_n * m0)**(3/2) * 1e-6  # cm^-3
Nv = (1/np.sqrt(2)) * (1/hbar**3) * (kT_J/np.pi * m_dos_p * m0)**(3/2) * 1e-6  # cm^-3

# Facteurs de normalisation pour la visualisation graphique (sans unités)
rho_norm_n = 0.8  # Pour que les courbes soient lisibles sur le graphique
rho_norm_p = 0.6

# Domaine d'énergie
E = np.linspace(-2, 2, 1000)

def rho_c(E):
    """
    Densité d'états dans la bande de conduction (normalisée pour visualisation)
    Forme: √(E - Ec) pour E ≥ Ec, 0 sinon
    """
    return np.where(E >= Ec, rho_norm_n * np.sqrt(E - Ec), 0)

def rho_v(E):
    """
    Densité d'états dans la bande de valence (normalisée pour visualisation)
    Forme: √(Ev - E) pour E ≤ Ev, 0 sinon
    """
    return np.where(E <= Ev, rho_norm_p * np.sqrt(Ev - E), 0)

def F_c(E, EFn):
    """
    Fonction de Fermi pour les électrons
    F_c(E) = 1 / (1 + exp((E - EFn)/kT))
    """
    return 1 / (1 + np.exp((E - EFn) / kT))

def F_v(E, EFp):
    """
    Fonction de Fermi pour les trous
    F_v(E) = 1 - F_c(E) = 1 / (1 + exp((EFp - E)/kT))
    """
    return 1 / (1 + np.exp((EFp - E) / kT))

def calculate_n_exact(EFn):
    """
    Calcule la concentration d'électrons avec l'approximation de Boltzmann
    n ≅ 2 × Nc × exp(-(Ec-EFn)/kT)
    """
    return 2 * Nc * np.exp(-(Ec - EFn) / kT)

def calculate_p_exact(EFp):
    """
    Calcule la concentration de trous avec l'approximation de Boltzmann
    p ≅ 2 × Nv × exp(-(EFp-Ev)/kT)
    """
    return 2 * Nv * np.exp(-(EFp - Ev) / kT)

def calculate_ni():
    """
    Calcule la concentration intrinsèque ni
    À l'équilibre EFn = EFp = EF au centre du gap
    """
    EF_intrinsic = (Ec + Ev) / 2  # EF au centre du gap pour le cas intrinsèque
    n_i = calculate_n_exact(EF_intrinsic)
    p_i = calculate_p_exact(EF_intrinsic)
    return np.sqrt(n_i * p_i)

def generate_ef_vs_logn_data():
    """
    Génère les données pour tracer EFn en fonction de log₁₀(n) et EFp en fonction de log₁₀(p)
    """
    # Plage de EF pour calculer la courbe complète
    EF_range = np.linspace(-1.5, 1.5, 300)
    n_values = []
    p_values = []
    
    for ef in EF_range:
        n = calculate_n_exact(ef)  # EFn = ef
        p = calculate_p_exact(ef)  # EFp = ef
        n_values.append(n)
        p_values.append(p)
    
    n_values = np.array(n_values)
    p_values = np.array(p_values)
    log10_n = np.log10(n_values)
    log10_p = np.log10(p_values)
    
    return EF_range, log10_n, log10_p

def generate_band_structure_data():
    """
    Génère les données pour les relations de dispersion E(k) des bandes
    """
    # Plage de vecteur d'onde k (en unités arbitraires pour la visualisation)
    k = np.linspace(-2, 2, 500)
    
    # Facteur de conversion pour avoir des énergies raisonnables (en eV)
    # Utilisons ℏ²/(2m₀) ≈ 3.81 eV·Å² pour normaliser
    energy_scale = 0.5  # Facteur d'échelle pour la visualisation
    
    # Relation de dispersion pour la bande de conduction
    # E_c(k) = Ec + ℏ²k²/(2m*_n)
    E_conduction = Ec + energy_scale * k**2 / m_dos_n
    
    # Relation de dispersion pour la bande de valence
    # E_v(k) = Ev - ℏ²k²/(2m*_p) (négative car les trous ont une masse effective positive)
    E_valence = Ev - energy_scale * k**2 / m_dos_p
    
    return k, E_conduction, E_valence

def plot_fermi_states(EFn, EFp):
    """
    Trace le diagramme complet avec trois graphiques:
    1. Relations de dispersion E(k)
    2. Densités d'états et fonctions de Fermi
    3. EF vs log₁₀(n) et log₁₀(p)
    
    Utilise EFn pour les électrons et EFp pour les trous
    """
    # Calcul des fonctions pour le graphique 2
    rho_c_vals = rho_c(E)
    rho_v_vals = rho_v(E)
    F_c_vals = F_c(E, EFn)  # Utilise EFn pour les électrons
    F_v_vals = F_v(E, EFp)  # Utilise EFp pour les trous
    
    # Produits pour les aires (visualisation)
    n_product = rho_c_vals * F_c_vals
    p_product = rho_v_vals * F_v_vals
    
    # Calcul des concentrations exactes
    n = calculate_n_exact(EFn)  # Concentration d'électrons avec EFn
    p = calculate_p_exact(EFp)  # Concentration de trous avec EFp
    ni = calculate_ni()
    
    # Produit n×p (différent de ni² si EFn ≠ EFp)
    np_product = n * p
    
    # Génération des données pour tous les graphiques
    EF_range, log10_n, log10_p = generate_ef_vs_logn_data()
    k, E_conduction, E_valence = generate_band_structure_data()
    
    # Fermer les figures précédentes
    plt.close('all')
    
    # Créer la figure avec trois sous-graphiques côte à côte
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(36, 10))
    
    # === GRAPHIQUE 0: Relations de dispersion E(k) ===
    # Tracer les bandes d'énergie
    ax0.plot(k, E_conduction, 'r-', linewidth=3, label='Bande de conduction')
    ax0.plot(k, E_valence, 'b-', linewidth=3, label='Bande de valence')
    
    # Lignes horizontales pour EFn et EFp
    ax0.axhline(y=EFn, color='red', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_{{Fn}}$ = {EFn:.3f} eV')
    ax0.axhline(y=EFp, color='blue', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_{{Fp}}$ = {EFp:.3f} eV')
    
    # Lignes horizontales pour Ec et Ev
    ax0.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1)
    ax0.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1)
    
    # Remplissage pour montrer les états occupés (approximation)
    # Zone occupée dans la bande de conduction (états en dessous de EFn)
    if EFn > Ec:
        k_limit_c = np.sqrt((EFn - Ec) * m_dos_n / 0.5)
        k_fill_c = k[np.abs(k) <= k_limit_c]
        E_fill_c = E_conduction[np.abs(k) <= k_limit_c]
        ax0.fill_between(k_fill_c, E_fill_c, EFn, 
                        where=(E_fill_c >= EFn), alpha=0.3, color='red',
                        label='États occupés (e⁻)')
    
    # Zone occupée dans la bande de valence (états en dessous de EFp)
    if EFp < Ev:
        k_limit_v = np.sqrt((Ev - EFp) * m_dos_p / 0.5)
        k_fill_v = k[np.abs(k) <= k_limit_v]
        E_fill_v = E_valence[np.abs(k) <= k_limit_v]
        ax0.fill_between(k_fill_v, E_fill_v, EFp,
                        where=(E_fill_v <= EFp), alpha=0.3, color='blue',
                        label='États libres (h⁺)')
    
    # Annotations des niveaux d'énergie
    ax0.text(1.7, Ec + 0.05, f'$E_c$ = {Ec:.3f} eV', fontsize=12, 
            ha='left', va='bottom', color='red', weight='bold')
    ax0.text(1.7, Ev - 0.05, f'$E_v$ = {Ev:.3f} eV', fontsize=12, 
            ha='left', va='top', color='blue', weight='bold')
    ax0.text(1.7, EFn + 0.1, f'$E_{{Fn}}$ = {EFn:.3f} eV', fontsize=12, 
            ha='left', va='center', color='red', weight='bold')
    ax0.text(1.7, EFp - 0.1, f'$E_{{Fp}}$ = {EFp:.3f} eV', fontsize=12, 
            ha='left', va='center', color='blue', weight='bold')
    
    # Configuration des axes pour le graphique 0
    ax0.set_xlabel('Vecteur d\'onde k (u.a.)', fontsize=12)
    ax0.set_ylabel('Énergie (eV)', fontsize=12)
    ax0.set_title(f'Relations de dispersion E(k)\nT = {T} K - Hors équilibre', fontsize=14)
    
    ax0.grid(True, alpha=0.3)
    ax0.legend(loc='upper right', fontsize=10)
    ax0.set_xlim(-2, 2)
    ax0.set_ylim(-1.5, 1.5)
    
    # === GRAPHIQUE 1: Densités d'états et fonctions de Fermi ===
    # Tracer les densités d'états
    ax1.plot(rho_c_vals, E, 'r-', linewidth=3, label=r'$\rho_c(E)$ (bande de conduction)')
    ax1.plot(rho_v_vals, E, 'b-', linewidth=3, label=r'$\rho_v(E)$ (bande de valence)')
    
    # Tracer les fonctions de Fermi
    ax1.plot(F_c_vals, E, 'r--', linewidth=2, alpha=0.8, label=r'$F_c(E, E_{Fn})$ (électrons)')
    ax1.plot(F_v_vals, E, 'b--', linewidth=2, alpha=0.8, label=r'$F_v(E, E_{Fp})$ (trous)')
    
    # Aires correspondant aux concentrations de porteurs
    mask_n = E >= Ec
    mask_p = E <= Ev
    
    ax1.fill_betweenx(E[mask_n], 0, n_product[mask_n], 
                     color='red', alpha=0.3, 
                     label=f'n = {n:.2e} cm⁻³')
    
    ax1.fill_betweenx(E[mask_p], 0, p_product[mask_p], 
                     color='blue', alpha=0.3, 
                     label=f'p = {p:.2e} cm⁻³')
    
    # Lignes de référence pour EFn et EFp
    ax1.axhline(y=EFn, color='red', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_{{Fn}}$ = {EFn:.3f} eV')
    ax1.axhline(y=EFp, color='blue', linestyle=':', linewidth=3, 
               alpha=0.8, label=f'$E_{{Fp}}$ = {EFp:.3f} eV')
    ax1.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1)
    ax1.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1)
    
    # Annotations des niveaux d'énergie
    ax1.text(1.4, Ec + 0.05, f'$E_c$ = {Ec:.3f} eV', fontsize=12, 
            ha='left', va='bottom', color='red', weight='bold')
    ax1.text(1.4, Ev - 0.05, f'$E_v$ = {Ev:.3f} eV', fontsize=12, 
            ha='left', va='top', color='blue', weight='bold')
    ax1.text(1.4, EFn + 0.1, f'$E_{{Fn}}$ = {EFn:.3f} eV', fontsize=12, 
            ha='left', va='center', color='red', weight='bold')
    ax1.text(1.4, EFp - 0.1, f'$E_{{Fp}}$ = {EFp:.3f} eV', fontsize=12, 
            ha='left', va='center', color='blue', weight='bold')
    
    # Configuration des axes pour le graphique 1
    ax1.set_xlabel('Densité d\'états / Probabilité d\'occupation', fontsize=12)
    ax1.set_ylabel('Énergie (eV)', fontsize=12)
    ax1.set_title(f'Densités d\'états et quasi-niveaux de Fermi\nApproximation de Boltzmann', fontsize=14)
    
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper right', fontsize=9)
    ax1.set_xlim(0, 1.6)
    ax1.set_ylim(-1.5, 1.5)
    
    # === GRAPHIQUE 2: EF vs log₁₀(n) et log₁₀(p) ===
    # Tracer la courbe EF vs log₁₀(n) en rouge
    ax2.plot(log10_n, EF_range, 'r-', linewidth=3, label=r'$E_{Fn}$ vs $\log_{10}(n)$')
    
    # Tracer la courbe EF vs log₁₀(p) en bleu
    ax2.plot(log10_p, EF_range, 'b-', linewidth=3, label=r'$E_{Fp}$ vs $\log_{10}(p)$')
    
    # Point correspondant à la valeur actuelle du slider pour n
    current_log10_n = np.log10(n)
    ax2.plot(current_log10_n, EFn, 'ro', markersize=8, 
             label=f'Point actuel (n)\n$E_{{Fn}}$ = {EFn:.3f} eV\n$\log_{{10}}(n)$ = {current_log10_n:.2f}')
    
    # Point correspondant à la valeur actuelle du slider pour p
    current_log10_p = np.log10(p)
    ax2.plot(current_log10_p, EFp, 'bo', markersize=8, 
             label=f'Point actuel (p)\n$E_{{Fp}}$ = {EFp:.3f} eV\n$\log_{{10}}(p)$ = {current_log10_p:.2f}')
    
    # Lignes horizontales pour EFn et EFp actuels
    ax2.axhline(y=EFn, color='red', linestyle=':', linewidth=2, alpha=0.8)
    ax2.axhline(y=EFp, color='blue', linestyle=':', linewidth=2, alpha=0.8)
    
    # Lignes verticales pour log₁₀(n) et log₁₀(p) actuels
    ax2.axvline(x=current_log10_n, color='red', linestyle='--', linewidth=1, alpha=0.6)
    ax2.axvline(x=current_log10_p, color='blue', linestyle='--', linewidth=1, alpha=0.6)
    
    # Lignes de référence pour les niveaux d'énergie
    ax2.axhline(y=Ec, color='red', linestyle='-', alpha=0.6, linewidth=1, label=f'$E_c$ = {Ec:.3f} eV')
    ax2.axhline(y=Ev, color='blue', linestyle='-', alpha=0.6, linewidth=1, label=f'$E_v$ = {Ev:.3f} eV')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.4, linewidth=1, label='Centre du gap')
    
    # Configuration des axes pour le graphique 2
    ax2.set_xlabel(r'$\log_{10}(n,p)$ [cm$^{-3}$]', fontsize=12)
    ax2.set_ylabel('Quasi-niveaux de Fermi (eV)', fontsize=12)
    ax2.set_title(r'Évolution des quasi-niveaux de Fermi', fontsize=14)
    
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper right', fontsize=9)
    ax2.set_ylim(-1.5, 1.5)
    
    # Ajuster automatiquement les limites de l'axe x pour bien voir les deux courbes
    all_log_values = np.concatenate([log10_n, log10_p])
    x_margin = (max(all_log_values) - min(all_log_values)) * 0.1
    ax2.set_xlim(min(all_log_values) - x_margin, max(all_log_values) + x_margin)
    
    # === INFORMATIONS SUR LE RÉGIME ===
    regime_text = determine_regime(EFn, EFp, n, p, ni, np_product)
    ax1.text(0.98, 0.02, regime_text, transform=ax1.transAxes, 
            fontsize=8, va='bottom', ha='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Informations supplémentaires pour le graphique 2
    info_text = f"n = {n:.2e} cm⁻³\n"
    info_text += f"p = {p:.2e} cm⁻³\n"
    info_text += f"$\log_{{10}}(n)$ = {current_log10_n:.2f}\n"
    info_text += f"$\log_{{10}}(p)$ = {current_log10_p:.2f}\n"
    info_text += f"$E_{{Fn}}$ = {EFn:.3f} eV\n"
    info_text += f"$E_{{Fp}}$ = {EFp:.3f} eV\n\n"
    info_text += f"$N_c$ = {Nc:.2e} cm⁻³\n"
    info_text += f"$N_v$ = {Nv:.2e} cm⁻³"
    
    ax2.text(0.02, 0.98, info_text, transform=ax2.transAxes, 
            fontsize=10, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8))
    
    # Informations pour le graphique 0 (relations de dispersion)
    band_info = f"Relations de dispersion:\n"
    band_info += f"$E_c(k) = E_c + \\frac{{\\hbar^2 k^2}}{{2m^*_n}}$\n"
    band_info += f"$E_v(k) = E_v - \\frac{{\\hbar^2 k^2}}{{2m^*_p}}$\n\n"
    band_info += f"$m^*_n = {m_dos_n}$ $m_0$\n"
    band_info += f"$m^*_p = {m_dos_p}$ $m_0$\n\n"
    band_info += f"Quasi-niveaux de Fermi:\n"
    band_info += f"$E_{{Fn}} - E_{{Fp}}$ = {EFn - EFp:.3f} eV"
    
    ax0.text(0.02, 0.98, band_info, transform=ax0.transAxes, 
            fontsize=10, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

def determine_regime(EFn, EFp, n, p, ni, np_product):
    """Détermine le régime du semiconducteur avec quasi-niveaux de Fermi"""
    gap_center = (Ec + Ev) / 2
    
    regime_info = f"n = {n:.2e} cm⁻³\n"
    regime_info += f"p = {p:.2e} cm⁻³\n"
    regime_info += f"n×p = {np_product:.2e} cm⁻⁶\n"
    regime_info += f"ni² = {ni**2:.2e} cm⁻⁶\n"
    regime_info += f"ni = {ni:.2e} cm⁻³\n\n"
    
    # Vérifier si le système est à l'équilibre ou hors équilibre
    if abs(EFn - EFp) < 0.01:  # Différence < 10 meV
        regime_info += "ÉQUILIBRE THERMIQUE\n"
        regime_info += f"(EFn ≈ EFp)\n"
        regime_info += f"n×p ≈ ni²\n\n"
        if abs(n - p) / max(n, p) < 0.1:
            regime_info += "Régime INTRINSÈQUE"
        elif n > p:
            regime_info += "Régime type N"
        else:
            regime_info += "Régime type P"
    else:
        regime_info += "HORS ÉQUILIBRE\n"
        regime_info += f"(EFn ≠ EFp)\n"
        regime_info += f"n×p ≠ ni²\n\n"
        if np_product > ni**2:
            regime_info += "Excès de porteurs\n"
            regime_info += f"n×p > ni²"
        else:
            regime_info += "Déficit de porteurs\n"
            regime_info += f"n×p < ni²"
    
    # Séparation des quasi-niveaux de Fermi
    splitting = abs(EFn - EFp)
    regime_info += f"\n\n|EFn - EFp| = {splitting:.3f} eV"
    regime_info += f"\nÉcart/kT = {splitting/kT:.1f}"
    
    return regime_info

# Créer les sliders interactifs pour les quasi-niveaux de Fermi
EFn_slider = widgets.FloatSlider(
    value=0.1,                    # Valeur initiale
    min=-1.5,                     # Valeur minimale
    max=1.5,                      # Valeur maximale
    step=0.01,                    # Pas
    description='$E_{Fn}$ (eV):',  # Description
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

EFp_slider = widgets.FloatSlider(
    value=-0.1,                   # Valeur initiale
    min=-1.5,                     # Valeur minimale
    max=1.5,                      # Valeur maximale
    step=0.01,                    # Pas
    description='$E_{Fp}$ (eV):',  # Description
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

# Calcul et affichage des paramètres initiaux
ni = calculate_ni()

# Afficher le graphique interactif
print("=== SIMULATION AVEC QUASI-NIVEAUX DE FERMI (HORS ÉQUILIBRE) ===")
print(f"Paramètres du semiconducteur (Silicium):")
print(f"• Gap d'énergie: Eg = {Eg:.2f} eV")
print(f"• Température: T = {T} K (kT = {kT:.4f} eV)")
print(f"• Masses effectives: m*n = {m_dos_n} m₀, m*p = {m_dos_p} m₀")
print(f"• Niveaux: Ec = {Ec:.3f} eV, Ev = {Ev:.3f} eV")
print(f"• Concentration intrinsèque: ni = {ni:.2e} cm⁻³")
print(f"• Densités effectives: Nc = {Nc:.2e} cm⁻³, Nv = {Nv:.2e} cm⁻³")
print(f"• Constantes: m₀ = {m0:.3e} kg, ℏ = {hbar:.3e} J·s")
print(f"\nFormules utilisées (approximation de Boltzmann valide):")
print(f"• n ≅ 2 × Nc × exp(-(Ec-EFn)/kT)")
print(f"• p ≅ 2 × Nv × exp(-(EFp-Ev)/kT)")
print(f"• HORS ÉQUILIBRE: n×p ≠ (2×ni)² si EFn ≠ EFp")
print(f"• ÉQUILIBRE: n×p = (2×ni)² si EFn = EFp = EF")
print("\nUtilisez les sliders pour observer:")
print("• Les quasi-niveaux de Fermi EFn et EFp dans tous les graphiques")
print("• L'évolution des concentrations n et p indépendamment")
print("• La violation de la loi d'action de masse: n×p ≠ ni² quand EFn ≠ EFp")
print("• Le retour à l'équilibre quand EFn = EFp")

widgets.interact(plot_fermi_states, EFn=EFn_slider, EFp=EFp_slider)

=== SIMULATION AVEC QUASI-NIVEAUX DE FERMI (HORS ÉQUILIBRE) ===
Paramètres du semiconducteur (Silicium):
• Gap d'énergie: Eg = 1.12 eV
• Température: T = 6000 K (kT = 0.5172 eV)
• Masses effectives: m*n = 1.08 m₀, m*p = 0.55 m₀
• Niveaux: Ec = 0.560 eV, Ev = -0.560 eV
• Concentration intrinsèque: ni = 1.03e+21 cm⁻³
• Densités effectives: Nc = 2.52e+21 cm⁻³, Nv = 9.15e+20 cm⁻³
• Constantes: m₀ = 9.109e-31 kg, ℏ = 1.055e-34 J·s

Formules utilisées (approximation de Boltzmann valide):
• n ≅ 2 × Nc × exp(-(Ec-EFn)/kT)
• p ≅ 2 × Nv × exp(-(EFp-Ev)/kT)
• HORS ÉQUILIBRE: n×p ≠ (2×ni)² si EFn ≠ EFp
• ÉQUILIBRE: n×p = (2×ni)² si EFn = EFp = EF

Utilisez les sliders pour observer:
• Les quasi-niveaux de Fermi EFn et EFp dans tous les graphiques
• L'évolution des concentrations n et p indépendamment
• La violation de la loi d'action de masse: n×p ≠ ni² quand EFn ≠ EFp
• Le retour à l'équilibre quand EFn = EFp


<>:284: SyntaxWarning: invalid escape sequence '\l'
<>:289: SyntaxWarning: invalid escape sequence '\l'
<>:327: SyntaxWarning: invalid escape sequence '\l'
<>:328: SyntaxWarning: invalid escape sequence '\l'
<>:284: SyntaxWarning: invalid escape sequence '\l'
<>:289: SyntaxWarning: invalid escape sequence '\l'
<>:327: SyntaxWarning: invalid escape sequence '\l'
<>:328: SyntaxWarning: invalid escape sequence '\l'
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4128518018.py:284: SyntaxWarning: invalid escape sequence '\l'
  label=f'Point actuel (n)\n$E_{{Fn}}$ = {EFn:.3f} eV\n$\log_{{10}}(n)$ = {current_log10_n:.2f}')
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4128518018.py:289: SyntaxWarning: invalid escape sequence '\l'
  label=f'Point actuel (p)\n$E_{{Fp}}$ = {EFp:.3f} eV\n$\log_{{10}}(p)$ = {current_log10_p:.2f}')
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_3879/4128518018.py:327: SyntaxWarning: invalid escape sequence '\l'
  info_

interactive(children=(FloatSlider(value=0.1, description='$E_{Fn}$ (eV):', layout=Layout(width='500px'), max=1…

<function __main__.plot_fermi_states(EFn, EFp)>